In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from tqdm import tqdm

In [32]:
data = pd.read_csv('datasets/categories2.csv')
data.head()

,Вопрос,Категория
0,Подписание контракта после отказа от заключени...,Заключение контракта
1,Участие в котировочной сессии,Участие в закупках
2,Страница контракта организации,Просмотр информации о контракте
3,Определение победителя и заключение контракта ...,Заключение контракта
4,Создание и публикация закупки с неизвестным об...,Создание закупки


In [33]:
data['Категория'].unique()

array(['Заключение контракта', 'Участие в закупках',
       'Просмотр информации о контракте', 'Создание закупки',
       'Технические вопросы', 'Работа с шаблонами',
       'Регистрация организации', 'Управление пользователями',
       'Уведомления', 'Техническая ошибка', 'Социальные сети',
       'Редактирование оферты/СТЕ', 'Просмотр информации об оферте',
       'Статусы оферты', 'Поиск информации', 'Статусы заявок',
       'Просмотр информации об организации', 'Просмотр информации',
       'Прямая закупка', 'Работа с файлами', 'Авторизация',
       'Электронный документооборот', 'Статусы контракта'], dtype=object)

In [34]:
X = data['Вопрос']
y = data['Категория']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [35]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Используемое устройство: {device}')

Используемое устройство: cuda


In [36]:
train_data = Dataset.from_pandas(pd.DataFrame({'text': X_train, 'label': y_train.factorize()[0]}))
test_data = Dataset.from_pandas(pd.DataFrame({'text': X_test, 'label': y_test.factorize()[0]}))

In [37]:
# 'distilbert-base-multilingual-cased'
# 'bert-base-multilingual-cased'
# 'xlm-roberta-base'
# 'distilbert-base-uncased'
# 'roberta-base'
# 'albert-base-v2'

model_name = 'distilbert-base-multilingual-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(y_train.unique())).to(device)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_test = test_data.map(tokenize_function, batched=True)

Map:   0%|          | 0/319 [00:00<?, ? examples/s]

Map:   0%|          | 0/80 [00:00<?, ? examples/s]

In [39]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
    report_to='none',
    fp16=torch.cuda.is_available()
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test
)

In [42]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,2.178800,3.085693
2,1.925200,3.116431
3,1.725600,3.127936


TrainOutput(global_step=60, training_loss=1.9972094853719076, metrics={'train_runtime': 85.404, 'train_samples_per_second': 11.206, 'train_steps_per_second': 0.703, 'total_flos': 126816516237312.0, 'train_loss': 1.9972094853719076, 'epoch': 3.0})

In [18]:
trainer.save_model('./models/class.model')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained('./models/class.model').to(device)

In [43]:
def classify_question_llm(question):
    inputs = tokenizer(question, return_tensors='pt', padding=True, truncation=True).to(device)
    outputs = model(**inputs)
    predicted_class = outputs.logits.argmax(dim=-1).item()
    return y_train.unique()[predicted_class]

In [49]:
data['Категория'].unique()

array(['Заключение контракта', 'Участие в закупках',
       'Просмотр информации о контракте', 'Создание закупки',
       'Технические вопросы', 'Работа с шаблонами',
       'Регистрация организации', 'Управление пользователями',
       'Уведомления', 'Техническая ошибка', 'Социальные сети',
       'Редактирование оферты/СТЕ', 'Просмотр информации об оферте',
       'Статусы оферты', 'Поиск информации', 'Статусы заявок',
       'Просмотр информации об организации', 'Просмотр информации',
       'Прямая закупка', 'Работа с файлами', 'Авторизация',
       'Электронный документооборот', 'Статусы контракта'], dtype=object)

In [44]:
example_question = 'Как изменить единицу измерения в оферте?'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Редактирование оферты/СТЕ


In [45]:
example_question = 'Что такое портал поставщиков'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Управление пользователями


In [46]:
example_question = 'Требования к изображению'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Просмотр информации


In [47]:
example_question = 'ошибка РДИК'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Техническая ошибка


In [48]:
example_question = 'Заявка отклонена без объяснения причин'
predicted_category = classify_question_llm(example_question)
print(f'Категория: {predicted_category}')

Категория: Техническая ошибка
